In [1]:
from __future__ import annotations
from pydantic import BaseModel, Field

from typing import Any, Dict, List
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent,
)

from langgraph.checkpoint.memory import MemorySaver

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.memory import BaseMemory

from GoogleCalender import CreateGoogleCalendarEvent
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)


In [2]:
class GetEventsSchema(BaseModel):
    start_datetime: str = Field(
        description=(
            "The start datetime for the event in the following format: "
            'YYYY-MM-DDTHH:MM:SS, where "T" separates the date and time.'
        )
    )
    end_datetime: str = Field(
        description=(
            "The end datetime for the event in the following format: "
            'YYYY-MM-DDTHH:MM:SS, where "T" separates the date and time.'
        )
    )
    agenda: str
    user_email: str

    timezone: str = Field(
        default="Asia/Kolkata",
    )

In [3]:
@tool
def identity(query):
    """This is the identity function return whatever user pass in query."""
    return query

In [4]:
credentials = get_gmail_credentials(
    token_file="../token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="credentials.json",
)

calendar_service = build_resource_service(
    credentials=credentials, service_name="calendar", service_version="v3"
)

createeventtool = CreateGoogleCalendarEvent.from_api_resource(calendar_service)

In [5]:
llm = ChatOpenAI()

In [6]:
def create_agent(llm, tools: list, system_prompt):

    agent = create_openai_tools_agent(llm, tools, system_prompt)
    executor = AgentExecutor(agent=agent, tools=tools)

    return executor

In [7]:
demo_prompt = """You are responsible for managing user queries related to demo requests.

Your primary role is to:

Politely greet the user and explain that you need to collect some basic information before proceeding.
Gather all necessary details from the user INCLUDING ANY THAT MAY HAVE BEEN FORGOTTEN like email id,date etc.
DO NOT MAKE ANY ASSUMPTION FOR EMAIL ID if not provided you should ask again for it but do not put by your self.
Convert the gathered information into the required format as outlined in {format_instructions}.
Use the formatted data to create an event in Google Calendar.
Confirm that the event has been created and scheduled at the requested time.
Provide the event link to the user.
"""

demo_parser = JsonOutputParser(pydantic_object=GetEventsSchema)

In [8]:
demo_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            demo_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ],
).partial(
    format_instructions=demo_parser.get_format_instructions(),
    # tools=[createeventtool]
)

In [9]:
demo = create_agent(
    llm=llm,
    tools=[createeventtool, identity],
    system_prompt=demo_prompt_template,
)

In [37]:
demo.invoke(
    {
        "messages": [
            HumanMessage(
                content="i want demo at 10:30 am to 11:30 am on 23nd august 2024",
            )
        ]
    }
)

{'messages': [HumanMessage(content='i want demo at 10:30 am to 11:30 am on 23nd august 2024')],
 'output': 'Great! I can help you schedule a demo. To proceed, I need the following details:\n\n1. Your email address\n2. Any specific agenda for the demo\n\nPlease provide the above information so we can schedule the demo for you.'}